# Inicializar bibliotecas

In [ ]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt


--2024-11-09 03:20:10--  https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt
Resolving emanuel3queijos.github.io (emanuel3queijos.github.io)... 185.199.111.153, 185.199.110.153, 185.199.108.153, ...
Connecting to emanuel3queijos.github.io (emanuel3queijos.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     691  --.-KB/s    in 0s      

2024-11-09 03:20:10 (35.0 MB/s) - ‘requirements.txt’ saved [691/691]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.

# Scripts para adicionar data dentro do banco



In [18]:
import psycopg2

# Função para criar a conexão
def create_connection():
    try:
        conn = psycopg2.connect(
            host="db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com",
            dbname="AWS-University-ETL-DB",
            user="postgres",
            password="Emanu0710**",
            port=5432
        )

        return conn
    except Exception as error:
        print(f"Erro ao conectar ao banco de dados: {error}")
        return None
create_connection()

<connection object at 0x7cd73b07e0c0; dsn: 'user=postgres password=xxx dbname=AWS-University-ETL-DB host=db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [19]:
def insert_cursos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            cursos = [
                ('Engenharia de Software', 'Curso que abrange o desenvolvimento de software, desde a análise de requisitos até a manutenção.'),
                ('Ciência da Computação', 'Curso focado nos fundamentos teóricos e práticos da computação e desenvolvimento de sistemas.'),
                ('Sistemas de Informação', 'Curso que combina TI com gestão, voltado para o desenvolvimento e gerenciamento de sistemas de informação.'),
                ('Análise e Desenvolvimento de Sistemas', 'Curso que prepara profissionais para projetar, desenvolver e implementar sistemas de software.'),
                ('Redes de Computadores', 'Curso voltado para a infraestrutura e comunicação entre computadores em redes.'),
                ('Inteligência Artificial', 'Curso focado em aprendizado de máquina, redes neurais, e outras áreas da IA.'),
                ('Engenharia de Controle e Automação', 'Curso voltado para automação industrial e sistemas de controle.'),
                ('Segurança da Informação', 'Curso que aborda a proteção de dados e sistemas contra ataques e acessos não autorizados.')
            ]

            for nome_curso, descricao in cursos:
                cur.execute("""
                    INSERT INTO tb_curso (nome, descricao)
                    VALUES (%s, %s)
                    RETURNING id_curso
                """, (nome_curso, descricao))

            conn.commit()
            print("Cursos inseridos com sucesso.")
        except Exception as error:
            print(f"Erro ao inserir cursos: {error}")
        finally:
            cur.close()
            conn.close()
insert_cursos()


Erro ao inserir cursos: column "nome" of relation "tb_curso" does not exist
LINE 2:                     INSERT INTO tb_curso (nome, descricao)
                                                  ^



# operações extras(deleta tudo não vei 😭)

In [ ]:

def export_to_csv(conn):
    try:
        cur = conn.cursor()

        tables = ["tb_aluno", "tb_curso", "tb_disciplina", "tb_turma", "tb_matricula", "tb_inscricao", "tb_turma_aluno"]
        for table in tables:
            with open(f'{table}.csv', 'w') as f:
                cur.copy_expert(f"COPY {table} TO STDOUT WITH CSV HEADER", f)

        cur.close()
        print("Dados exportados com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao exportar dados: {error}")
    finally:
        if conn is not None:
            conn.close()


#Export tables
export_to_csv()

In [ ]:
# Limpar as tabelas
def clean_tables(conn):
    try:
        cur = conn.cursor()
        truncate_sql = """TRUNCATE TABLE tb_turma_aluno, tb_inscricao, tb_matricula, tb_turma, tb_disciplina, tb_curso, tb_aluno RESTART IDENTITY CASCADE;"""
        cur.execute(truncate_sql)
        conn.commit()
        print("Tabelas limpas com sucesso")
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao truncar tabelas: {error}")

    finally:
        if conn is not None:
            conn.close()

clean_tables()

Tabelas limpas com sucesso
